In [22]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [4]:
dataframe = pd.read_csv("club_data.csv")

In [8]:
dataframe.drop(columns=['Timestamp'] , axis = 1, inplace = True)

In [11]:
dataframe.columns

Index(['Name', 'Domain', 'Role in your domain (Web Developer , Management)',
       'Linkedin Profile Link  (If you dont have Linkedin profile , Type randomly)',
       'Instagram Link (If you dont have instagram profile , Type randomly)',
       'Email (rknec.edu)',
       'GitHub Link( (If you dont have Github profile , Type randomly)'],
      dtype='object')

In [15]:
dataframe.rename(columns={
    'Name': 'name', 
    'Domain': 'domain',
    'Role in your domain (Web Developer , Management)' : 'role',
    'Linkedin Profile Link  (If you dont have Linkedin profile , Type randomly)':'linkedin',
    'Instagram Link (If you dont have instagram profile , Type randomly)' : 'instagram',
    'Email (rknec.edu)' : 'email',
    'GitHub Link( (If you dont have Github profile , Type randomly)' : 'github'
    }, inplace = True)

In [23]:
dataframe.head()

,name,domain,role,linkedin,instagram,email,github
0,Abhay Mishra,Web Dev,Web Developer,https://www.linkedin.com/in/abhaymishra1371?ut...,https://www.instagram.com/_abhay_mishra206?igs...,mishraar_2@rknec.edu,NaN
1,Tisha Parmar,Socials,"Content Writer, Socials",https://www.linkedin.com/in/tisha-parmar-76992...,tishaparmar_04,Parmarts@rknec.edu,NaN
2,Harsh Pashine,Cloud & AI,Cloud Developer,https://www.linkedin.com/in/harsh-pashine-434a...,qwertyuiop,pashinehd@rknec.edu,NaN
3,Jiya vij,Graphics,Designer,https://www.linkedin.com/in/jiya-vij-871b42292...,https://www.instagram.com/jiya_vij?igsh=MTNnZz...,vijj@rknec.edu,NaN
4,Devyani Keche,Management,Management,https://www.linkedin.com/in/devyani-keche,https://www.instagram.com/devyani_keche?igsh=M...,kecheds@rknec.edu,NaN


In [26]:
db_user = 'postgres'      # Replace with your PostgreSQL username
db_password = 'Mongapost$2341'  # Replace with your PostgreSQL password
db_host = 'localhost'             # Replace with your PostgreSQL host (usually localhost)
db_port = '5432'                  # Replace with your PostgreSQL port (usually 5432)
db_name = 'explore'    # Replace with your database name

db_connection_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
db_engine = create_engine(db_connection_str)

In [27]:
dataframe.to_sql('members', db_engine, if_exists='fail', index=False)

19

In [ ]:
# Minimum guaranteed time for each speaker (in minutes)
MIN_TIME = 15

# Total session time available (in minutes)
TOTAL_TIME = 120

# Domains (these map to the speakers/topics)
domains = [
    "design", "database_backend", "android", "webdevelopment",
    "competitive_programming", "machine_learning", "cloud"
]

# Input - Percentage responses for each option (you will provide these manually)
percentages = {
    "q1_yes": 80,  # chosen CS by choice
    "q1_no": 20,   # forced into CS
    "q2_9to5": 30,
    "q2_business": 50,
    "q2_notsure": 20,
    "q3_logical": 70,
    "q3_creative": 30,
    "q4_step_by_step": 40,
    "q4_explore": 50,
    "q4_theory": 10,
    "q5_backend": 65,
    "q5_frontend": 35,
    "q6_india": 50,
    "q6_usa": 40,
    "q6_dubai": 10
}

# Domain weights based on percentages
domain_weights = {
    "design": (
        percentages["q1_no"] * 2 +
        percentages["q2_business"] +
        percentages["q3_creative"] * 3 +
        percentages["q4_explore"] * 2 +
        percentages["q5_frontend"] +
        percentages["q6_dubai"] * 2
    ),
    "database_backend": (
        percentages["q1_yes"] * 2 +
        percentages["q2_9to5"] * 2 +
        percentages["q3_logical"] * 2 +
        percentages["q4_step_by_step"] * 2 +
        percentages["q5_backend"] * 3
    ),
    "android": (
        percentages["q4_explore"] * 2 +
        percentages["q6_dubai"] * 2
    ),
    "webdevelopment": (
        percentages["q1_yes"] +
        percentages["q2_business"] * 2 +
        percentages["q4_step_by_step"] +
        percentages["q5_frontend"] * 3
    ),
    "competitive_programming": (
        percentages["q2_notsure"] * 2 +
        percentages["q3_logical"] * 2 +
        percentages["q6_india"] * 2
    ),
    "machine_learning": (
        percentages["q1_yes"] +
        percentages["q2_notsure"] +
        percentages["q4_theory"] * 2 +
        percentages["q6_usa"] * 2
    ),
    "cloud": (
        percentages["q2_9to5"] +
        percentages["q5_backend"] +
        percentages["q6_usa"]
    )
}

# Minimum guaranteed time for all speakers
time_allocation = {domain: MIN_TIME for domain in domains}

# Time left after assigning minimums
remaining_time = TOTAL_TIME - len(domains) * MIN_TIME

# Total weight to distribute remaining time
total_weight = sum(domain_weights.values())

# Distribute remaining time proportionally based on weight
for domain, weight in domain_weights.items():
    if total_weight > 0:
        extra_time = (weight / total_weight) * remaining_time
        time_allocation[domain] += int(extra_time)

# Final Time Allocation Report
print("\n===== Final Speaker Time Allocation Report =====")
for domain, time in time_allocation.items():
    print(f"{domain.capitalize()}: {time} minutes")




===== Final Speaker Time Allocation Report =====
Design: 17 minutes
Database_backend: 19 minutes
Android: 15 minutes
Webdevelopment: 17 minutes
Competitive_programming: 17 minutes
Machine_learning: 16 minutes
Cloud: 16 minutes


In [3]:
def calculate_speaker_times_from_percentages(percentage_data):
    """
    Calculates speaker times for different domains based on percentage data for each survey option.

    Args:
        percentage_data: A dictionary where keys are question numbers ('Q1', 'Q2', etc.)
                         and values are dictionaries of options and their corresponding percentages.
                         Example:
                         {
                             'Q1': {'yes': 60, 'no': 40},  # 60% said yes, 40% said no to Q1
                             'Q2': {'9 to 5 job': 50, 'self business': 30, 'not sure': 20},
                             ...
                         }

    Returns:
        A dictionary containing speaker times for each domain (in minutes)
        and a string representing a report.
    """

    domains = {
        "design": 0,
        "database_backend": 0,
        "android": 0,
        "web_development": 0,
        "competitive_programming": 0,
        "machine_learning": 0,
        "cloud": 0,
    }

    # Question 1: CS by choice?
    q1_percentages = percentage_data.get('Q1', {})  # Get percentages for Q1, default to empty dict if Q1 not found
    if 'yes' in q1_percentages:
        percentage_yes = q1_percentages['yes'] / 100.0  # Convert percentage to fraction
        for domain in domains:
            domains[domain] += 0.1 * percentage_yes
    if 'no' in q1_percentages:
        percentage_no = q1_percentages['no'] / 100.0
        # No score impact for 'no' in Q1 based on initial logic, but you could add a negative impact if needed

    # Question 2: Job preference
    q2_percentages = percentage_data.get('Q2', {})
    if '9 to 5 job' in q2_percentages:
        percentage_9to5 = q2_percentages['9 to 5 job'] / 100.0
        domains['database_backend'] += 0.2 * percentage_9to5
        domains['android'] += 0.1 * percentage_9to5
        domains['web_development'] += 0.1 * percentage_9to5
        domains['cloud'] += 0.1 * percentage_9to5
    if 'self business' in q2_percentages:
        percentage_self = q2_percentages['self business'] / 100.0
        domains['design'] += 0.1 * percentage_self
        domains['web_development'] += 0.1 * percentage_self
        domains['cloud'] += 0.1 * percentage_self
        domains['machine_learning'] += 0.1 * percentage_self
    if 'not sure' in q2_percentages:
        percentage_notsure = q2_percentages['not sure'] / 100.0
        # No score impact for 'not sure' in Q2 based on initial logic, but you could add if needed


    # Question 3: Logical or Creative
    q3_percentages = percentage_data.get('Q3', {})
    if 'logical' in q3_percentages:
        percentage_logical = q3_percentages['logical'] / 100.0
        domains['database_backend'] += 0.3 * percentage_logical
        domains['competitive_programming'] += 0.3 * percentage_logical
        domains['machine_learning'] += 0.2 * percentage_logical
        domains['cloud'] += 0.2 * percentage_logical
    if 'creative' in q3_percentages:
        percentage_creative = q3_percentages['creative'] / 100.0
        domains['design'] += 0.3 * percentage_creative
        domains['web_development'] += 0.2 * percentage_creative
        domains['android'] += 0.2 * percentage_creative

    # Question 4: Learning preference
    q4_percentages = percentage_data.get('Q4', {})
    if 'step by step process' in q4_percentages:
        percentage_stepwise = q4_percentages['step by step process'] / 100.0
        domains['database_backend'] += 0.2 * percentage_stepwise
        domains['competitive_programming'] += 0.2 * percentage_stepwise
        domains['android'] += 0.1 * percentage_stepwise
    if 'experiment and explore' in q4_percentages:
        percentage_experiment = q4_percentages['experiment and explore'] / 100.0
        domains['design'] += 0.2 * percentage_experiment
        domains['web_development'] += 0.2 * percentage_experiment
        domains['cloud'] += 0.1 * percentage_experiment
        domains['machine_learning'] += 0.1 * percentage_experiment
    if 'theory and principle' in q4_percentages:
        percentage_theory = q4_percentages['theory and principle'] / 100.0
        domains['database_backend'] += 0.2 * percentage_theory
        domains['competitive_programming'] += 0.2 * percentage_theory
        domains['machine_learning'] += 0.2 * percentage_theory
        domains['cloud'] += 0.2 * percentage_theory

    # Question 5: Backend or Frontend
    q5_percentages = percentage_data.get('Q5', {})
    if 'backend' in q5_percentages:
        percentage_backend = q5_percentages['backend'] / 100.0
        domains['database_backend'] += 0.5 * percentage_backend
    if 'frontend' in q5_percentages:
        percentage_frontend = q5_percentages['frontend'] / 100.0
        domains['design'] += 0.3 * percentage_frontend
        domains['web_development'] += 0.2 * percentage_frontend
        domains['android'] += 0.2 * percentage_frontend

    # Question 6: Work Location - (Ignoring for time calculation as per initial thought process)
    q6_percentages = percentage_data.get('Q6', {})
    if 'USA' in q6_percentages:
        percentage_usa = q6_percentages['USA'] / 100.0
        domains['machine_learning'] += 0.3 * percentage_usa  # Increased weightage
        domains['cloud'] += 0.3 * percentage_usa         # Increased weightage
    if 'India' in q6_percentages:
        percentage_india = q6_percentages['India'] / 100.0
        domains['web_development'] += 0.2 * percentage_india # Increased weightage (Frontend proxy)
        domains['database_backend'] += 0.2 * percentage_india # Increased weightage (Data & Backend)
        domains['android'] += 0.2 * percentage_india       # Increased weightage
    if 'Dubai' in q6_percentages:
        percentage_dubai = q6_percentages['Dubai'] / 100.0
        domains['database_backend'] += 0.2 * percentage_dubai # Increased weightage
        domains['design'] += 0.2 * percentage_dubai        # Increased weightage

    total_score = sum(domains.values())
    if total_score == 0:
        normalized_domains = {domain: 1/len(domains) for domain in domains} # If no score, distribute equally
    else:
        normalized_domains = {domain: score / total_score for domain, score in domains.items()}

    base_time = 15 # minutes
    total_event_time = 180
    remaining_time = total_event_time - (len(domains) * base_time)
    if remaining_time < 0:
        remaining_time = 0

    speaker_times = {}
    for domain, normalized_score in normalized_domains.items():
        extra_time = normalized_score * remaining_time
        speaker_times[domain] = base_time + extra_time
        speaker_times[domain] = max(15, speaker_times[domain])

    
    report = "Speaker Time Allocation Report (Based on Percentage Data):\n\n"
    for domain, time in speaker_times.items():
        report += f"- {domain.replace('_', ' ').title()}: {time:.2f} minutes\n"

    report += "\nThis report is generated based on the percentage data of survey responses to help plan speaker time allocation. "
    report += "Each speaker is allocated a minimum of 15 minutes."

    return speaker_times, report


percentage_data = {
    'Q1': {'yes': 70, 'no': 30},
    'Q2': {'9 to 5 job': 60, 'self business': 25, 'not sure': 15},
    'Q3': {'logical': 55, 'creative': 45},
    'Q4': {'step by step process': 40, 'experiment and explore': 35, 'theory and principle': 25},
    'Q5': {'backend': 65, 'frontend': 35},
    'Q6': {'India': 75, 'USA': 15, 'Dubai': 10} 
}


speaker_times, report = calculate_speaker_times_from_percentages(percentage_data)

print("Speaker Times:", speaker_times)
print("\nReport:\n", report)

Speaker Times: {'design': 24.06827880512091, 'database_backend': 35.9103840682788, 'android': 25.24182076813656, 'web_development': 26.415362731152204, 'competitive_programming': 22.78805120910384, 'machine_learning': 22.147937411095306, 'cloud': 23.428165007112376}

Report:
 Speaker Time Allocation Report (Based on Percentage Data):

- Design: 24.07 minutes
- Database Backend: 35.91 minutes
- Android: 25.24 minutes
- Web Development: 26.42 minutes
- Competitive Programming: 22.79 minutes
- Machine Learning: 22.15 minutes
- Cloud: 23.43 minutes

This report is generated based on the percentage data of survey responses to help plan speaker time allocation. Each speaker is allocated a minimum of 15 minutes.
